# Investigates A03C1, A06S1, Met Station pits poor performance

In [ ]:
import pandas as pd
import numpy as np
import scipy.io as sio
from copy import deepcopy

In [ ]:
from smrt_aesop_setup import model, trenchdata, return_snowpack, sensor_89

In [ ]:
from run_smrt_with_realistic_atmosphere import get_atmosphere, tb_results

# A03C1

### Investigate low surface density

In [ ]:
profile = eval('trenchdata.A03C1')
# Get atmosphere
atmosphere = get_atmosphere('C087', 'A03', single_frequency=True)
snowpack = return_snowpack(profile, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere)

In [ ]:
results = model.run(sensor_89, snowpack)

In [ ]:
tb_results(results.TbV(), single_frequency=89e9)

In [ ]:
alt_profile = deepcopy(profile)
alt_profile.mean_layer_densityA.mean_layer_densityA[0] = 104
# Replace surface density with default value
alt_snowpack = return_snowpack(alt_profile, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere)
alt_results = model.run(sensor_89, alt_snowpack)

In [ ]:
alt_snowpack.layers[0].density

In [ ]:
tb_results(alt_results.TbV(), single_frequency=89e9)

### Investigate high SS SSA

In [ ]:
profile_ss_ssa = deepcopy(profile)
# Replace SS with nan so uses default value
profile_ss_ssa.mean_layer_SSA1.mean_layer_SSA1[0] = np.nan
# Create snowpack
snowpack_ss_nan = return_snowpack(profile_ss_ssa, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere)

In [ ]:
# Run model
tb_results(model.run(sensor_89, snowpack_ss_nan).TbV(), single_frequency=89e9)

# A06S1

### Due to low WS SSA?

In [ ]:
# First create snowpack with observed WS SSA
profile = eval('trenchdata.A06S1')
atmosphere_A06 = get_atmosphere('C087', 'A06', single_frequency=True)

snowpack_A06S1 = return_snowpack(profile, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A06)

# Modify data to assume no observations of WS SSA made
# Therefore default value used instead
profile_with_nan = deepcopy(profile)
profile_with_nan.mean_layer_SSA1.mean_layer_SSA1[1] = np.nan
# Create new snowpack
snowpack_nan_WS_SSA = return_snowpack(profile_with_nan, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A06)

In [ ]:
# Run model on snowpacks
result = model.run(sensor_89, snowpack_A06S1)
result_with_nan = model.run(sensor_89, snowpack_nan_WS_SSA)

In [ ]:
print(tb_results(result.TbV(), single_frequency=89e9), tb_results(result_with_nan.TbV(), single_frequency=89e9))

In [ ]:
# What if all SSA were nan?
profile_all_ssa_nan = deepcopy(profile)
profile_all_ssa_nan.mean_layer_SSA1.mean_layer_SSA1 = [np.nan]*3
snowpack_all_nan = return_snowpack(profile_all_ssa_nan, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A06)
tb_results(model.run(sensor_89, snowpack_all_nan).TbV(), single_frequency=89e9)


# Met Station

### NB this was taken after the storm, so could look different to airborne data

### Met Station has high SSA in DH - what happens if default value is used?

In [ ]:
profile = eval('trenchdata.MetS')
atmosphere_A04 = get_atmosphere('C087', 'A04', single_frequency=True)

snowpack = return_snowpack(profile, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A04)

# SNOWPACK DH nan
# Therefore default value used instead
profile_with_dh_nan = deepcopy(profile)
profile_with_dh_nan.mean_layer_SSA1.mean_layer_SSA1[2] = np.nan
# Create new snowpack
snowpack_nan_dh_SSA = return_snowpack(profile_with_dh_nan, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A04)

# Snowpack WS nan
# Therefore default value used instead
profile_with_ws_nan = deepcopy(profile)
profile_with_ws_nan.mean_layer_SSA1.mean_layer_SSA1[1] = np.nan
# Create new snowpack
snowpack_nan_ws_SSA = return_snowpack(profile_with_ws_nan, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A04)

# What if all SSA were nan?
profile_all_ssa_nan = deepcopy(profile)
profile_all_ssa_nan.mean_layer_SSA1.mean_layer_SSA1 = [np.nan]*3
snowpack_all_nan = return_snowpack(profile_all_ssa_nan, with_ice_lens=True, dh_debye=1.2,
                           flight_temperature_correction='C087',
                           atmosphere=atmosphere_A04)

In [ ]:
# Run model on snowpacks
result = model.run(sensor_89, snowpack)
result_with_dh_nan = model.run(sensor_89, snowpack_nan_dh_SSA)
result_with_ws_nan = model.run(sensor_89, snowpack_nan_ws_SSA)
result_all_nan = model.run(sensor_89, snowpack_all_nan)

In [ ]:
print(tb_results(result.TbV(), single_frequency=89e9))
print(tb_results(result_with_dh_nan.TbV(), single_frequency=89e9))
print(tb_results(result_with_ws_nan.TbV(), single_frequency=89e9))
print(tb_results(result_all_nan.TbV(), single_frequency=89e9))